In [11]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [12]:
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())

In [13]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2022-01-27 20:51:00--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2022-01-27 20:51:01--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6c94fede463f10eea52d68006e.dl.dropboxusercontent.com/cd/0/get/BeluM6-hBvvNC8DzZugmSDl_Jy6yA_Q5ZCmW3WoHhX9EPqagUdDxE24-Xftsvo5HAilRUas3b6pFFmUMX147Z2wR5SQruQfuFqPvfyWTHZ0pIX1lZNsW9twnkvgyvGoMmyViaQ3K-u6ui6LiyPWEKBNq/file?dl=1# [following]
--2022-01-27 20:51:01--  https://uc6c94fede463f10eea52d68006e.dl.dropboxusercontent.com/cd/0/get/BeluM6-hBvvNC8DzZugmSDl_Jy6yA_Q5ZCmW3WoHhX9EPqagUdDxE24-Xftsvo5HAilRUas3b6pFFmUM

In [14]:
!ls stocks.parquet -l

-rw-rw-r-- 1 ubuntu ubuntu 678640589 Jan 27 20:51 stocks.parquet


In [15]:
data=pd.read_parquet('stocks.parquet')

In [17]:
smdata = data[['date', 'adj_close', 'symbol']]
smdata['date']= pd.to_datetime(smdata['date'])

dates = smdata.groupby('symbol', as_index=False).agg(first_date=pd.NamedAgg(column='date', aggfunc="min"), last_date=pd.NamedAgg(column='date', aggfunc="max"))
dates.head(2)
 
diff = dates.merge(smdata.rename(columns={'adj_close':'first_adj_close'}), left_on=['symbol','first_date'],right_on=['symbol','date']).drop('date', axis=1)
diff = diff.merge(smdata.rename(columns={'adj_close':'last_adj_close'}), left_on=['symbol', 'last_date'], right_on=['symbol','date']).drop('date', axis=1)
diff['days']= (diff.last_date - diff.first_date).dt.days
diff.head(3)

diff['return']=diff.apply(lambda row: None if row.first_adj_close == 0 or row.days == 0 else row.last_adj_close/row.first_adj_close/row.days, axis=1)

diff = diff.sort_values('return', ascending=False)
diff = diff.head(20)
diff

/tmp/ipykernel_1878/2902733303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smdata['date']= pd.to_datetime(smdata['date'])


,symbol,first_date,last_date,first_adj_close,last_adj_close,days,return
7126,NVO,1981-04-30,2021-12-27,2.275794e-09,111.629997,14851,3.302876e+06
6925,NKE,1980-12-02,2021-12-27,3.775067e-06,167.580002,15000,2.959418e+03
2420,DCI,1980-03-17,2021-12-27,9.812100e-06,57.939999,15260,3.869564e+02
9747,TJX,1987-06-26,2021-12-27,2.857077e-05,74.290001,12603,2.063167e+02
10765,WTRG,1980-03-17,2021-12-27,4.768529e-04,52.549999,15260,7.221605e+00
9932,TTC,1980-03-17,2021-12-27,1.051490e-03,99.879997,15260,6.224708e+00
10164,USB,1973-05-03,2021-12-27,7.828808e-04,56.740002,17770,4.078555e+00
6292,MCD,1966-07-05,2021-12-27,3.923203e-03,268.239990,20264,3.374097e+00
7729,PII,1987-09-16,2021-12-27,3.345692e-03,107.000000,12521,2.554223e+00
4565,HD,1981-09-22,2021-12-27,1.823626e-02,404.089996,14706,1.506773e+00
